In [ ]:
import os
import sys

# Standardize working directory to project root
try:
    if os.path.basename(os.getcwd()) in ['modeling', 'scraping', 'data-prep', 'analysis', 'setup', 'misc']:
        os.chdir('../..')
    elif os.path.basename(os.getcwd()) == 'notebooks':
        os.chdir('..')
        
    if os.getcwd() not in sys.path:
        sys.path.append(os.getcwd())
        
    print(f"Working Directory set to: {os.getcwd()}")
except Exception as e:
    print(f"Failed to set working directory: {e}")

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import mynnlib
from mynnlib import *

dataset_dir = "insect-dataset/binary-classifier"

# Dataset

In [43]:
for class_name in os.listdir(f"{dataset_dir}/data"):
    cnt = sum([1 for class_name in os.listdir(f"{dataset_dir}/data/{class_name}")])
    print(f"{class_name:10} : {cnt}")

insect     : 822
other      : 955


In [44]:
for class_name in os.listdir(f"{dataset_dir}/val"):
    cnt = sum([1 for class_name in os.listdir(f"{dataset_dir}/val/{class_name}")])
    print(f"{class_name:10} : {cnt}")

insect     : 30
other      : 13


# Train

### Model A (resnet-18)

In [45]:
training_params = [
    { "idx": 1, "robustness": 0.2, "break_at_val_acc_diff": 0.05},
    { "idx": 2, "robustness": 0.5, "break_at_val_acc_diff": 0.02},
    { "idx": 3, "robustness": 1.0, "break_at_val_acc_diff": 0.01},
    { "idx": 4, "robustness": 2.0, "break_at_val_acc_diff": -0.000001},
    { "idx": 5, "robustness": 2.0, "break_at_val_acc_diff": -0.000001},
    { "idx": 6, "robustness": 2.0, "break_at_val_acc_diff": -0.000001}
]
for param in training_params:
    print(f"Phase {param["idx"]}:")
    if param["idx"] == 1:
        model_data = init_model_for_training(f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                             batch_size=32, arch="resnet18", image_size=224, robustness=param["robustness"],
                                             lr=1e-4, weight_decay=1e-4, silent=True)
    else:
        model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                            batch_size=32, image_size=224, robustness=param["robustness"], silent=True)
    train(model_data, 5, f"{dataset_dir}/checkpoint.root-classifier.ta.ep{param["idx"]:02}###.pth", 
          break_at_val_acc_diff=param["break_at_val_acc_diff"])

Phase 1:
Epoch    1 /    5  | Train Loss: 0.1991 Acc: 0.9173  | Val Loss: 0.2988 Acc: 0.8837  | Elapsed time: 0:00:11.891865
Epoch    2 /    5  | Train Loss: 0.0477 Acc: 0.9842  | Val Loss: 0.5127 Acc: 0.7907  | Elapsed time: 0:00:25.734035
Phase 2:
Epoch    1 /    5  | Train Loss: 0.2478 Acc: 0.8936  | Val Loss: 0.5369 Acc: 0.8140  | Elapsed time: 0:00:14.541735
Epoch    2 /    5  | Train Loss: 0.1892 Acc: 0.9257  | Val Loss: 0.7013 Acc: 0.7907  | Elapsed time: 0:00:30.192471
Phase 3:
Epoch    1 /    5  | Train Loss: 0.2061 Acc: 0.9218  | Val Loss: 0.5354 Acc: 0.7674  | Elapsed time: 0:00:15.359614
Epoch    2 /    5  | Train Loss: 0.1681 Acc: 0.9364  | Val Loss: 0.5301 Acc: 0.8140  | Elapsed time: 0:00:31.023119
Epoch    3 /    5  | Train Loss: 0.1523 Acc: 0.9381  | Val Loss: 1.4553 Acc: 0.7674  | Elapsed time: 0:00:46.645951
Phase 4:
Epoch    1 /    5  | Train Loss: 0.1564 Acc: 0.9353  | Val Loss: 0.6374 Acc: 0.7907  | Elapsed time: 0:00:15.620752
Epoch    2 /    5  | Train Loss: 0.1

In [52]:
model_data = torch.load(f"{dataset_dir}/checkpoint.root-classifier.ta.ep060000.pth", weights_only=False)

In [53]:
test_class(model_data, f"{dataset_dir}/val", ['insect', 'other'])

Class: insect          ----> Success: 30/30 -> 100.00%
Class: other           ----> Success: 7/13 -> 53.85%


In [54]:
test_top_k(model_data, f"{dataset_dir}/test", 1, print_genus_match=False, print_no_match=True, print_preds=False)

other-04 (hotdog): 
	 insect(0.931)  
other-06 (flower): 
	 insect(0.841)  
other-07 (rock): 
	 insect(0.987)  
other-09 (oriole): 
	 insect(0.952)  
other-10 (dog): 
	 insect(0.957)  
other-12 (kingfisher): 
	 insect(0.514)  
Top   1 accuracy: 38/44 -> 86.36%
Top   1 accuracy: 38/44 -> 86.36%


### Model B (resnet-34) ***

In [51]:
training_params = [
    { "idx": 1, "robustness": 0.2, "break_at_val_acc_diff": 0.05},
    { "idx": 2, "robustness": 0.5, "break_at_val_acc_diff": 0.02},
    { "idx": 3, "robustness": 1.0, "break_at_val_acc_diff": 0.01},
    { "idx": 4, "robustness": 2.0, "break_at_val_acc_diff": -0.000001},
    { "idx": 5, "robustness": 2.0, "break_at_val_acc_diff": -0.000001},
    { "idx": 6, "robustness": 2.0, "break_at_val_acc_diff": -0.000001}
]
for param in training_params:
    print(f"Phase {param["idx"]}:")
    if param["idx"] == 1:
        model_data = init_model_for_training(f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                             batch_size=32, arch="resnet34", image_size=224, robustness=param["robustness"],
                                             lr=1e-4, weight_decay=1e-4, silent=True)
    else:
        model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                            batch_size=32, image_size=224, robustness=param["robustness"], silent=True)
    train(model_data, 5, f"{dataset_dir}/checkpoint.root-classifier.tb.ep{param["idx"]:02}###.pth", 
          break_at_val_acc_diff=param["break_at_val_acc_diff"])

Phase 1:
Epoch    1 /    5  | Train Loss: 0.1847 Acc: 0.9150  | Val Loss: 0.1013 Acc: 0.9535  | Elapsed time: 0:00:14.834023
Epoch    2 /    5  | Train Loss: 0.0582 Acc: 0.9797  | Val Loss: 0.2177 Acc: 0.8837  | Elapsed time: 0:00:30.416462
Phase 2:
Epoch    1 /    5  | Train Loss: 0.2567 Acc: 0.8953  | Val Loss: 0.5157 Acc: 0.8140  | Elapsed time: 0:00:16.635585
Epoch    2 /    5  | Train Loss: 0.1752 Acc: 0.9302  | Val Loss: 0.3952 Acc: 0.8372  | Elapsed time: 0:00:33.741523
Epoch    3 /    5  | Train Loss: 0.1595 Acc: 0.9381  | Val Loss: 0.2087 Acc: 0.9302  | Elapsed time: 0:00:51.494270
Epoch    4 /    5  | Train Loss: 0.1508 Acc: 0.9370  | Val Loss: 0.4280 Acc: 0.8372  | Elapsed time: 0:01:09.263364
Phase 3:
Epoch    1 /    5  | Train Loss: 0.1680 Acc: 0.9330  | Val Loss: 0.4454 Acc: 0.8372  | Elapsed time: 0:00:17.794616
Epoch    2 /    5  | Train Loss: 0.1407 Acc: 0.9443  | Val Loss: 0.3492 Acc: 0.8837  | Elapsed time: 0:00:35.971152
Epoch    3 /    5  | Train Loss: 0.1209 Acc: 

In [70]:
model_data = torch.load(f"{dataset_dir}/checkpoint.root-classifier.tb.ep060001.pth", weights_only=False)

In [71]:
test_class(model_data, f"{dataset_dir}/val", ['insect', 'other'])

Class: insect          ----> Success: 29/30 -> 96.67%
Class: other           ----> Success: 10/13 -> 76.92%


In [72]:
test_top_k(model_data, f"{dataset_dir}/test", 1, print_genus_match=False, print_no_match=True, print_preds=False)

insect-1: 
	 other(0.802)  
other-07 (rock): 
	 insect(0.999)  
other-09 (oriole): 
	 insect(0.891)  
other-10 (toad): 
	 insect(0.601)  
Top   1 accuracy: 40/44 -> 90.91%
Top   1 accuracy: 40/44 -> 90.91%
